In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split


In [2]:
DATASET_PATH = "Dataset"
MODEL_PATH = "pushup_cnn_lstm.h5"
IMG_SIZE = 64
EPOCHS = 10
BATCH_SIZE = 8

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
# def load_video_frames_from_folder(folder, label):
#     frames = []
#     labels = []
#     for filename in os.listdir(folder):
#         file_path = os.path.join(folder, filename)
#         cap = cv2.VideoCapture(file_path)
#         frame_list = []
#         while True:
#             ret, frame = cap.read()
#             if not ret:
#                 break
#             frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
#             frame = img_to_array(frame) / 255.0
#             frame_list.append(frame)
#         cap.release()
#         if len(frame_list) > 0:
#             frames.append(frame_list)
#             labels.append(label)
#     return frames, labels

# def load_dataset():
#     print("📂 Loading dataset...")
#     correct_frames, correct_labels = load_video_frames_from_folder(os.path.join(DATASET_PATH, "Correct sequence"), 1)
#     wrong_frames, wrong_labels = load_video_frames_from_folder(os.path.join(DATASET_PATH, "Wrong sequence"), 0)

#     frames = correct_frames + wrong_frames
#     labels = correct_labels + wrong_labels

#     # Pad/truncate sequences to same length
#     max_len = max(len(f) for f in frames)
#     X = np.zeros((len(frames), max_len, IMG_SIZE, IMG_SIZE, 3))
#     for i, seq in enumerate(frames):
#         for j in range(min(len(seq), max_len)):
#             X[i, j] = seq[j]
#     y = np.array(labels)
#     print("✅ Dataset loaded successfully.")
#     return X, y

TARGET_FRAMES = 60   # or whatever number you want

def reduce_frames(frame_list, target_frames=TARGET_FRAMES):
    total = len(frame_list)
    indices = np.linspace(0, total - 1, target_frames).astype(int)
    reduced = [frame_list[i] for i in indices]
    return np.array(reduced)


def load_video_frames_from_folder(folder, label):
    frames, labels = [], []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        cap = cv2.VideoCapture(file_path)
        extracted = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frame = img_to_array(frame) / 255.0
            extracted.append(frame)

        cap.release()

        if len(extracted) >= 5:
            frames.append(reduce_frames(extracted))
            labels.append(label)

    return frames, labels


# def load_dataset():
#     print("📂 Loading dataset...")

#     correct_frames, correct_labels = load_video_frames_from_folder(
#         os.path.join(DATASET_PATH, "Correct sequence"), 1)

#     wrong_frames, wrong_labels = load_video_frames_from_folder(
#         os.path.join(DATASET_PATH, "Wrong sequence"), 0)

#     X = np.array(correct_frames + wrong_frames)
#     y = np.array(correct_labels + wrong_labels)

#     print("New X shape:", X.shape)  # (N, 30, 64, 64, 3)
#     print("Unique labels:", set(y))

#     print("✅ Dataset loaded successfully.")
#     return X, y



def load_dataset():
    print("📂 Loading dataset...")

    # Load raw frames
    correct_frames, correct_labels = load_video_frames_from_folder(
        os.path.join(DATASET_PATH, "Correct sequence"), 1)

    wrong_frames, wrong_labels = load_video_frames_from_folder(
        os.path.join(DATASET_PATH, "Wrong sequence"), 0)

    frames = correct_frames + wrong_frames
    labels = correct_labels + wrong_labels

    # -----------------------------
    # ⚠️ DYNAMIC FRAME HANDLING
    # -----------------------------
    # If model exists → get expected frame count from model
    if os.path.exists(MODEL_PATH):
        print("🧠 Pretrained model detected → using its input sequence length.")
        model = load_model(MODEL_PATH)
        max_len = model.input_shape[1]     # LSTM expected timesteps
        print(f"Model expects {max_len} frames.")
    else:
        print("📌 No pretrained model → using longest video sequence.")
        max_len = max(len(f) for f in frames)
        print(f"Max frame length found: {max_len}")

    # -----------------------------
    # PAD / TRUNCATE EACH VIDEO
    # -----------------------------
    X = np.zeros((len(frames), max_len, IMG_SIZE, IMG_SIZE, 3))

    for i, seq in enumerate(frames):
        seq_len = len(seq)

        if seq_len >= max_len:
            # truncate extra frames
            X[i] = np.array(seq[:max_len])
        else:
            # pad remaining frames with zeros
            X[i, :seq_len] = np.array(seq)

    y = np.array(labels)

    print("New X shape:", X.shape)
    print("Unique labels:", set(y))
    print("✅ Dataset loaded successfully.\n")

    return X, y



In [ ]:
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=input_shape),
        TimeDistributed(MaxPooling2D((2, 2))),

        TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
        TimeDistributed(MaxPooling2D((2, 2))),

        TimeDistributed(Flatten()),
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.4),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


In [5]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.utils import shuffle   # ✅ FIX: Now imported properly


def evaluate_model(model, X_test, y_test):
    print("\n📊 Evaluating Model...")

    # Predict
    y_pred_raw = model.predict(X_test).reshape(-1)
    y_pred = (y_pred_raw > 0.5).astype(int)

    print("\nAccuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, zero_division=0))
    print("Recall:", recall_score(y_test, y_pred, zero_division=0))
    print("F1 Score:", f1_score(y_test, y_pred, zero_division=0))

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))


# MAIN PIPELINE
if os.path.exists(MODEL_PATH):
    print("🧠 Pre-trained model found! Loading existing model...")
    model = load_model(MODEL_PATH)
    print("✅ Model loaded successfully.")

    X, y = load_dataset()
    X, y = shuffle(X, y, random_state=42)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    evaluate_model(model, X_test, y_test)

else:
    print("⚙️ Training new model...")

    X, y = load_dataset()
    X, y = shuffle(X, y, random_state=42)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # ✅ FIX: Use dynamic frame length instead of undefined TARGET_FRAMES
    TARGET_FRAMES = X_train.shape[1]

    input_shape = (TARGET_FRAMES, IMG_SIZE, IMG_SIZE, 3)

    model = create_cnn_lstm_model(input_shape)

    model.fit(
        X_train,
        y_train,
        validation_data=(X_test, y_test),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE
    )

    model.save(MODEL_PATH)
    print("✅ Model saved!")

    evaluate_model(model, X_test, y_test)


🧠 Pre-trained model found! Loading existing model...
✅ Model loaded successfully.
📂 Loading dataset...


🧠 Pretrained model detected → using its input sequence length.
Model expects 20 frames.
New X shape: (100, 20, 64, 64, 3)
Unique labels: {0, 1}
✅ Dataset loaded successfully.


📊 Evaluating Model...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 692ms/step


ValueError: Found input variables with inconsistent numbers of samples: [20, 40]

In [ ]:
import tkinter as tk
from tkinter import messagebox

def start_detection():
    messagebox.showinfo("Starting", "Real-time push-up detection starting...")
    run_detection()

def show_metrics():
    y_pred_raw = model.predict(X_test)
    y_pred = (y_pred_raw > 0.5).astype(int)
    msg = classification_report(y_test, y_pred)
    messagebox.showinfo("Model Performance", msg)


root = tk.Tk()
root.title("AI Push-up Trainer")
root.geometry("400x300")

tk.Button(root, text="Start Real-Time Detection", command=start_detection).pack(pady=20)
tk.Button(root, text="Show Model Metrics", command=show_metrics).pack(pady=20)
tk.Button(root, text="Exit", command=root.destroy).pack(pady=20)

root.mainloop()


In [ ]:
def calculate_angle(a, b, c):
    a = np.array(a); b = np.array(b); c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - \
              np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180:
        angle = 360 - angle
    return angle


def run_detection():
    cap = cv2.VideoCapture(0)
    pose = mp_pose.Pose()

    counter = 0
    stage = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark

            # Right arm joints
            shoulder = [lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                     lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                     lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)

            # Push-up logic
            if angle < 90:
                stage = "down"
            if angle > 160 and stage == "down":
                stage = "up"
                counter += 1

            cv2.putText(frame, f'Reps: {counter}', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 3)

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow("AI Push-Up Trainer", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
